In [1]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from pylab import *
rc('axes', linewidth=3)

In [2]:
"""
Setting up the parameters of the RFMEO
"""
lamda = [1-i/10 for i in range(9)]#list of free parameters
p = len(lamda)-1#Number of Sites
l = 2#length

In [9]:
"""
Generating the Training and Test Sets
"""

"""
Training Set Generation
"""
h = 0.1#training step size
t = np.arange(0,1+h, h)
Xtr = t.reshape(1,len(t))
n = Xtr.shape[1]#Number of training samples

"""
Testing Set Generation
"""
h = 0.0001#testing step size
t0 = np.arange(0, 0.9+h, h)
Xte = t0.reshape(1, len(t0))
nprime = Xte.shape[1]#Number of testing samples


In [4]:
"""
Governing system of ODE
"""

scale = 10000# Scaling parameter for the system of ode

def systemfunctions(y_hat, lamda, l):
    f = (y_hat).copy()
    if l>1:
        z1 = 0
        z2 = 0
        for j in range(l):
            z1 += y_hat[j]
            z2 += y_hat[j+1]
        f[0] = (lamda[0]*(1-z1)-lamda[1]*y_hat[0]*(1-z2))
        for i in range(1,p-l):
            z1 = 0
            z2 = 0
            for j in range(i,i+l):
                z1 += y_hat[j]
                z2 += y_hat[j+1]
            f[i] = lamda[i]*y_hat[i-1]*(1-z1)-lamda[i+1]*(y_hat[i])*(1-z2)
        z1 = 0
        for j in range(p-l,p):
            z1 += y_hat[j]
        f[p-l] = lamda[p-l]*y_hat[p-l-1]*(1-z1)-lamda[p-l+1]*y_hat[p-l]
        for i in range(p-l+1,p):
            f[i] = lamda[i]*y_hat[i-1]-lamda[i+1]*y_hat[i]
    else:
        f = (y_hat).copy()
        f[0] = lamda[0]*(1-y_hat[0])-lamda[1]*y_hat[0]*(1-y_hat[1])
        for i in range(1,p-1):
            f[i] = lamda[i]*y_hat[i-1]*(1-y_hat[i])-lamda[i+1]*(y_hat[i])*(1-y_hat[i+1])
        f[p-1] = lamda[p-1]*y_hat[p-2]*(1-y_hat[p-1])-lamda[p]*y_hat[p-1]
    return scale*f

In [22]:
"""
Setting Up the Neural Network for ODE
"""
def init_weights_biases(initialiser, N0, N1):
    if initialiser.upper() == 'NORMAL':
        return [np.random.normal(0,np.sqrt(2/N1),(N1,N0)), np.random.normal(0, np.sqrt(2/N1), (N1,1))]
    if initialiser.upper() == 'UNIFORM':
        return [np.random.uniform(0,0.05,(N1,N0)),np.random.uniform(0,0.05,(N1,1))]
    if initialiser.upper() == 'XAVIER':
        return [np.random.uniform(0, 1/np.sqrt(N1), (N1,N0)), np.random.normal(0, 1/np.sqrt(N1), (N1,1))]
    else:
        return -1


def add_layer(input_shape, hidden_units , activation = 'sigmoid', initialiser = 'normal'): 
    weights_biases = init_weights_biases(initialiser, input_shape, hidden_units)
    NA.append(activation)
    NW.append(weights_biases[0])
    NB.append(weights_biases[1])
    return None    


def activation_function(x, string, alpha = 0.01):
    if string.upper() == 'SIGMOID':
        return (1/(1+np.exp(-x)))
    if string.upper() == 'BPS':
        return 2*(1/(1+np.exp(-x)))-1
#     if string.upper() == 'BPS':
#         return (1-np.exp(-x))/(1+np.exp(-x))
    if string.upper() == 'SIN':
        return np.sin(3*x)
    if string.upper()=='COS':
        return np.cos(3*x)
    if string.upper() == 'CUSTOM':
        return x+np.sin(x)**2
    if string.upper() == 'TANH':
        return np.tanh(x)
    if string.upper() == 'RELU':
        y = (x)/2+np.abs(x)/2
        return y
    if string.upper() == 'LEAKYRELU' or string.upper() == 'LR':
        return (x+alpha*x+np.abs(x-alpha*x))/2
    if string.upper() == 'LINEAR':
        return x
    if string.upper() == 'EXPONENTIAL' or string.upper() == 'EXP':
        return np.exp(x)
    if string.upper() == 'ELU':
        x[x<0] = 0.01*(np.exp(x[x<0])-1)
        return x


def activation_derivative(x, string, alpha = 0.01):
    if string.upper() == 'SIGMOID':
        return (1/(1+np.exp(-x)))*(1-(1/(1+np.exp(-x))))
    if string.upper() == 'BPS':
        return 2*(1/(1+np.exp(-x)))*(1-(1/(1+np.exp(-x))))
    if string.upper() == 'SIN':
        return 3*np.cos(3*x)
    if string.upper() == 'COS':
        return -3*np.sin(3*x)
    if string.upper() == 'TANH':
        return (1-np.tanh(x)**2)
    if string.upper() == "RELU":
        x[x<=0] = 0
        x[x!=0] = 1
        return x
    if string.upper() == 'CUSTOM':
        y = 1+2*np.sin(x)*np.cos(x)
        return y
    if string.upper() == 'LEAKYRELU' or string.upper() == 'LR':
        dx = np.ones(x.shape)
        dx[x < 0] = alpha
        x = dx.copy()
        return x
    if string.upper() == 'LINEAR':
        x = 1
        return x
    if string.upper() == 'EXPONENTIAL' or string.upper() == 'EXP':
        return np.exp(x)
    if string.upper() == 'ELU':
        x[x>=0] = 1
        x[x<0] = 0.01*(np.exp(x[x<0]))
        return x
    else:
        return -1
    

def forward_propagation(X,NA,NW,NB):
    A = [X]
    Z = []
    for i in range(len(NA)):
        Zstar = (NW[i]@A[i]+NB[i])
        Astar = activation_function(Zstar.astype(float), NA[i])
        Z.append(Zstar.astype(float))
        A.append(Astar)
    return([Z,A])


def rmsprop(NW, NB, dW, dB, SW, SB, epsilon, lr, beta):
    for i in range(L):
        SW[i] = (beta*SW[i]+(1-beta)*dW[i]**2)
        SB[i] = (beta*SB[i]+(1-beta)*dB[i]**2)
        NW[i] = NW[i]-lr*dW[i]/(SW[i]**0.5+epsilon)
        NB[i] = NB[i]-lr*dB[i]/(SB[i]**0.5+epsilon)
    return [NW, NB, SW, SB]


def adam(i, NW, NB, dW, dB, VW, VB, SW, SB, epsilon, lr, momentum, beta):
    VWhat = VW.copy()
    VBhat = VB.copy()
    SWhat = SW.copy()
    SBhat = SB.copy()
    for j in range(L):
        VW[j] = momentum*VW[j]+(1-momentum)*dW[j]
        VB[j] = momentum*VB[j]+(1-momentum)*dB[j]
        SW[j] = beta*SW[j]+(1-beta)*(dW[j]**2)
        SB[j] = beta*SB[j]+(1-beta)*(dB[j]**2)
        VWhat[j] = VW[j]/(1-momentum**i)
        VBhat[j] = VB[j]/(1-momentum**i)
        SWhat[j] = SW[j]/(1-beta**i)
        SBhat[j] = SB[j]/(1-beta**i)
        NW[j] = NW[j]-lr*(VWhat[j]/np.sqrt(SWhat[j]+epsilon))
        NB[j] = NB[j]-lr*(VBhat[j]/np.sqrt(SBhat[j]+epsilon))
    return [NW, NB, VW, VB, SW, SB]
   
def backward_propagation(NA, NW, Z, A, dZ, dW, dB, y_hat, f, l):
    Adot = (np.gradient(A[L], 0.1)[0])/scale-10**-10
    Adotdot = (np.gradient(Adot, 0.1)[0])/scale
    dAL = [0 for i in range(p)]
    if l>1:#if l>1
        z2 = 0
        penalty = 1
        for j in range(l):
            z2 += y_hat[j+1]
        df0a0 = -lamda[0]*t*np.ones_like(A[L][0])-lamda[1]*t*np.ones_like(A[L][0])*(1-z2)
        df1a0 = lamda[1]*t*np.ones_like(A[L][0])*(1-z2)
        dAL[0] = (A[L][0]+t*Adot[0]-f[0])*(1+t*Adotdot[0]/Adot[0]-df0a0)-(A[L][1]+t*Adot[1]-f[1])*df1a0

        z2 = 0
        for j in range(l):
            z2 += y_hat[j+2]
        df0a1 = -lamda[0]*t*np.ones_like(A[L][1])+lamda[1]*y_hat[0]*t*np.ones_like(A[L][1])
        df1a1 = -lamda[1]*y_hat[0]*t*np.ones_like(A[L][1])-lamda[2]*t*np.ones_like(A[L][1])*(1-z2)
        df2a1 = lamda[2]*t*np.ones_like(A[L][1])*(1-z2)
        dAL[1] = -(A[L][0]+t*Adot[0]-f[0])*df0a1+(A[L][1]+t*Adot[1]-f[1])*(1+t*Adotdot[1]/Adot[1]-df1a1)-(A[L][2]+t*Adot[2]-f[2])*df2a1

        for i in range(2, l):
            index = 1
            dfa1L = [0 for j in range(i+2)]
            dfa1R = [0 for j in range(i+2)]
            dfa1L[0] = -lamda[0]*t*np.ones_like(A[L][i])
            dfa1R[0] = lamda[1]*y_hat[0]*t*np.ones_like(A[L][i])
            for j in range(1,len(dfa1R)-2):
                    dfa1L[j] = -dfa1R[j-1]
                    dfa1R[j] = lamda[index+1]*y_hat[index]*t*np.ones_like(A[L][i])
                    index += 1
            dfa1L[len(dfa1R)-2] = -dfa1R[len(dfa1R)-3]
            z2 = 0
            for k in range(l):
                z2 += y_hat[i+k+1]
            dfa1R[len(dfa1R)-2] = -lamda[index+1]*t*np.ones_like(A[L][i])*(1-z2)
            dfa1L[len(dfa1R)-1] = -dfa1R[len(dfa1R)-2]
            for o in range(len(dfa1R)):
                if o!=len(dfa1R)-2:
                    dAL[i] += -(A[L][o]+t*Adot[o]-f[o])*(dfa1L[o]+dfa1R[o])
                if o == len(dfa1R)-2:
                    dAL[i] += (A[L][o]+t*Adot[o]-f[o])*(1+t*Adotdot[o]/Adot[o]-(dfa1L[o]+dfa1R[o]))
        inc = 0
        for i in range(l,p-l):
            dfa4L = [0 for i in range(l+2)]
            dfa4R = [0 for i in range(l+2)]
            dfa4R[0] = lamda[i-l+1]*y_hat[i-l]*t
            for k in range(1,l):
                dfa4L[k] = -dfa4R[k-1]
                dfa4R[k] = lamda[i-l+k+1]*y_hat[i-l+k]*t*np.ones_like(A[L][i])
            dfa4L[l] = -dfa4R[l-1]
            z2 = 0
            for s in range(i,l+i):
                z2 += y_hat[s+1] 
            dfa4R[l] = -lamda[i+1]*t*np.ones_like(A[L][i])*(1-z2)
            dfa4L[l+1] = -dfa4R[l]
            for r in range(l+2):
                if r!= l:
                    dAL[i] += -(A[L][r+inc]+t*Adot[r+inc]-f[r+inc])*(dfa4L[r]+dfa4R[r])
                if r == l:
                    dAL[i] += (A[L][r+inc]+t*Adot[r+inc]-f[r+inc])*(1+t*Adotdot[r+inc]/Adot[r+inc]-(dfa4L[r]+dfa4R[r]))
            inc += 1
        dfa7R = [0 for i in range(l+2)]
        dfa7L = [0 for i in range(l+2)]
        dfa7R[0] = lamda[p-2*l+1]*y_hat[p-2*l]*t*np.ones_like(A[L][p-l])
        for j in range(1,l):
            dfa7L[j] = -dfa7R[j-1]
            dfa7R[j] = lamda[p-2*l+1+j]*y_hat[p-2*l+j]*t*np.ones_like(A[L][p-l])
        dfa7L[l] = -dfa7R[l-1]
        dfa7R[l] = -lamda[p-l+1]*t*np.ones_like(A[L][p-l]) 
        dfa7L[l+1] = -dfa7R[l]
        for r in range(l+2):
                if r!= l:
                    dAL[p-l] += -(A[L][r+inc]+t*Adot[r+inc]-f[r+inc])*(dfa7L[r]+dfa7R[r])
                if r == l:
                    dAL[p-l] += (A[L][r+inc]+t*Adot[r+inc]-f[r+inc])*(1+t*Adotdot[r+inc]/Adot[r+inc]-(dfa7L[r]+dfa7R[r]))
        inc += 1
        q = 0
        for i in range(p-l+1,p-1):
            dfa8L = [0 for i in range(l+2-q)]
            dfa8R = [0 for i in range(l+2-q)]
            q += 1
            dfa8R[0] = lamda[i-l+1]*y_hat[i-l]*t*np.ones_like(A[L][i])
            for k in range(1,len(dfa8R)-3):
                    dfa8L[k] = -dfa8R[k-1]
                    dfa8R[k] = lamda[i-l+k+1]*y_hat[i-l+k]*t*np.ones_like(A[L][i])
            dfa8L[k+1] = -dfa8R[k]
            dfa8R[k+2] = -lamda[i+1]*t*np.ones_like(A[L][i])
            dfa8L[k+3] = -dfa8R[k+2]
            for r in range(len(dfa8R)-2):
                dAL[i] += -(A[L][r+inc]+t*Adot[r+inc]-f[r+inc])*(dfa8L[r]+dfa8R[r])
            dAL[i] = dAL[i]+(A[L][i]+t*Adot[i]-f[i])*(1+t*Adotdot[i]/Adot[i]-(dfa8L[r+1]+dfa8R[r+1]))-(A[L][i+1]+t*Adot[i+1]-f[i+1])*(dfa8L[r+2]+dfa8R[r+2])
            inc += 1
        df6a10 = +lamda[p-l]*y_hat[p-l-1]*t*np.ones_like(A[L][p-1])
        df7a10 = -df6a10
        df10a10 = -lamda[p]*t*np.ones_like(A[L][p-1])
        dAL[p-1] = -(Adot[p-l-1]+t*Adot[p-l-1]-f[p-l-1])*df6a10-(A[L][p-l]+t*Adot[p-l]-f[p-l])*df7a10+(A[L][p-1]+t*Adot[p-1]-f[p-1])*(1+t*Adotdot[p-1]/Adot[p-1]-df10a10)
    #     print(np.array(dAL).shape)
    
    else:# if l = 1
        df0a0 = -lamda[0]*t-lamda[1]*t*(1-y_hat[1])
        df1a0 = lamda[1]*t*(1-y_hat[1])
        dAL[0] = (A[L][0]+t*Adot[0]-f[0])*(1+t*Adotdot[0]/Adot[0]-df0a0)-(A[L][1]+t*Adot[1]-f[1])*df1a0

        for i in range(1,p-1):
            df0a1 = lamda[i]*y_hat[i-1]*t
            df1a1 = -df0a1-lamda[i+1]*t*(1-y_hat[i+1])
            df2a1 = lamda[i+1]*t*(1-y_hat[i+1])
            dAL[i] = (A[L][i-1]+t*Adot[i-1]-f[i-1])*df0a1+(A[L][i]+t*Adot[i]-f[i])*(1+t*Adotdot[i]/Adot[i]-df1a1)-(A[L][i+1]+t*Adot[i+1]-f[i+1])*df2a1

        dfp1ap = lamda[p-1]*y_hat[p-2]*t
        dfpap = -dfp1ap-lamda[p]*t
        dAL[p-1] = -(A[L][p-2]+t*Adot[p-2]-f[p-2])*dfp1ap+(A[L][p-1]+t*Adot[p-1]-f[p-1])*(1+t*Adotdot[p-1]/Adot[p-1]-dfpap)
    dtAL = np.array(dAL).reshape(p,n)/n
    for i in range(L-1,-1,-1):    
        dZ[i] = dtAL*activation_derivative(Z[i],NA[i])
        dW[i] = (dZ[i]@A[i].T)/n
        dB[i] = np.sum(dZ[i], axis = 1, keepdims = True)/n
        dtAL = (NW[i].T@dZ[i])/n
    return [dZ, dW, dB]



def train_model(X, epochs, NA, NW, NB, optimiser, learning_rate = 0.001, 
                momentum = 0.9, epsilon = 10**-8, beta = 0.99):
    [dZ, dW, dB] = [[0 for i in range(L)],[0 for i in range(L)],[0 for i in range(L)]]
    VW = [np.zeros(NW[i].shape) for i in range(L)]
    VB = [np.zeros(NB[i].shape) for i in range(L)]
    SW = [np.zeros(NW[i].shape) for i in range(L)]
    SB = [np.zeros(NB[i].shape) for i in range(L)]
    for i in range(epochs):
        [Z, A] = forward_propagation(X, NA, NW, NB)
        yhat = init+t*(A[L])
        f = systemfunctions(yhat, lamda, l)
        [dZ, dW, dB] = backward_propagation(NA, NW, Z, A, dZ, dW, dB, yhat, f, l)
        if optimiser.upper() == 'RMSPROP':
            [NW, NB, SW, SB] =  rmsprop(NW, NB, dW, dB, SW, SB, epsilon, learning_rate, beta)
        elif optimiser.upper() == 'ADAM':
            [NW, NB, VW, VB, SW, SB] = adam(i+1, NW, NB, dW, dB, VW, VB, SW, SB, epsilon, learning_rate, momentum, beta)
        if (i/epochs)*100 in range(100):
            print('█', end = '')
#     print('\n')
    return [NW, NB]

In [25]:
"""
Training and Testing the Network
"""
ensemblesize = 6#Number of Medians
L = 3# Number of Hidden Layers
for i in range(ensemblesize):
    plt.figure(figsize = (12,10))
    print(i+1, end = '.')
    init = np.random.uniform(0, 1/l, (p,1))
    [NA, NW, NB] = [[],[],[]]
    add_layer(input_shape = 1, hidden_units = p, activation = 'tanh', initialiser = 'normal')
    for e in range(L-2):
        add_layer(input_shape = p, hidden_units = p, activation = 'sigmoid', initialiser = 'xavier')
    add_layer(input_shape = p, hidden_units = p, activation = 'bps', initialiser = 'xavier')
    """
    Training the CDNN on the train set
    """
    [NW, NB] = train_model(Xtr, 3000, NA, NW, NB, optimiser = 'rmsprop',
                                     learning_rate = 10**-2, momentum = 0.9, beta = 0.999)
    """
    Obtaining the solutions on the test set
    """
    [Z,A] = forward_propagation(Xte, NA, NW, NB)
    vhat = init+Xte*A[L]
    for j in range(p):
        plt.plot(Xte.ravel(), vhat[j])
    plt.show()
    print('\n')